<a href="https://colab.research.google.com/github/RDGopal/IB9AU-2026/blob/main/L4_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###***Make sure you have your API keys for Gemini and HuggingFace***



## Overview of LLMs: Proprietary vs. Open Source

Large Language Models (LLMs) are a type of artificial intelligence program capable of generating human-like text, understanding context, and performing various language-related tasks. They are trained on vast amounts of text data.

LLMs can generally be categorized into two main types:

### 1. Proprietary Models

These are models developed and owned by private companies. They are often closed-source, meaning their internal architecture, training data, and weights are not publicly accessible. Users typically interact with these models through APIs provided by the developers.

**Characteristics:**
*   **Controlled Access:** Access is usually via API keys or paid subscriptions.
*   **High Performance:** Often trained on massive datasets with significant computational resources, leading to state-of-the-art performance.
*   **Ease of Use:** Generally well-documented APIs and managed infrastructure make them easy to integrate and use.
*   **Less Transparency:** The inner workings are typically opaque.
*   **Examples:** Google's Gemini, OpenAI's GPT series, Anthropic's Claude.

### 2. Open Source Models

These models are released with their code, weights, and sometimes training data publicly available. This allows researchers and developers to inspect, modify, and deploy them freely.

**Characteristics:**
*   **Transparency:** Full access to the model's architecture and weights encourages research and understanding.
*   **Customization:** Can be fine-tuned or adapted for specific tasks and datasets.
*   **Community Support:** Benefit from a vibrant community that contributes improvements, tools, and documentation.
*   **Deployment Flexibility:** Can be hosted on various infrastructures, from local machines to cloud services, giving users more control over data privacy and costs.
*   **Examples:** HuggingFace's Transformers library models (e.g., GPT-2, Llama, Falcon), Mistral, Gemma.

This notebook demonstrates how to interact with both proprietary models (specifically Google's Gemini) and open-source models (using HuggingFace Transformers).

#Part 1 - Proprietary Models

### API Key Setup

To interact with proprietary models like Google Gemini, you often need an API key. This key authenticates your requests and manages your usage. In Google Colab, it's good practice to store sensitive information like API keys in `Userdata` secrets for security.

*   `import os` and `from google.colab import userdata`: These lines import necessary modules for interacting with the operating system and Colab's user data, respectively.
*   `os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')`: This retrieves your API key, named 'GEMINI_API_KEY', from Colab secrets and sets it as an environment variable.
*   `genai.configure(api_key=os.environ["GOOGLE_API_KEY"])`: This configures the `google.generativeai` library with your API key, allowing you to make authenticated requests to the Gemini API.

In [ ]:
# 1. Setup (Use the standard library)
# Get your API key from https://aistudio.google.com/

import os
from google.colab import userdata
import google.generativeai as genai

os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

### Listing Available Models

The `genai.list_models()` function allows you to see all the available Generative AI models that you can access through the API. The loop then filters these models to display only those that support the `generateContent` method, which indicates they are suitable for text generation tasks (LLMs).

In [ ]:
print("Available LLM Models:")
# Iterate through all models and filter
for m in genai.list_models():
    # 'generateContent' indicates it is a text-generation model (LLM)
    if 'generateContent' in m.supported_generation_methods:
        print(f"- {m.name}")

In [ ]:
from IPython.display import display, Markdown

### Basic Text Generation

This section demonstrates a basic text generation request using the Gemini API:

1.  `model = genai.GenerativeModel("gemini-flash-latest")`: This line initializes a `GenerativeModel` object, specifying the particular Gemini model you want to use (`gemini-flash-latest` in this case).
2.  `response = model.generate_content(...)`: This makes a call to the model to generate content. You pass a list of strings as input (your prompt).
3.  `print(response.text)`: The generated text content from the model's response is then printed.

In [ ]:
# Ask the Question
model = genai.GenerativeModel("gemini-flash-latest")

response = model.generate_content(
    [
        "Explain what AI is"
    ]
)

print("\n--- ANSWER ---")
print(response.text)

In [ ]:
display(Markdown(response.text))

### Controlling Generation Parameters

When generating content, you can often control aspects of the output using `generation_config` parameters:

*   `"temperature"`: This parameter controls the randomness of the output. A higher temperature (e.g., 1.0) leads to more creative and diverse responses, while a lower temperature (e.g., 0.1-0.5) makes the output more deterministic and focused. Here, `0.7` provides a moderate level of creativity.
*   `"max_output_tokens"`: This parameter sets a limit on the number of tokens (words or sub-word units) the model will generate in its response. This is useful for controlling response length and managing API costs. Here, the output is capped at `100` tokens.

In [ ]:
# Ask the Question
model = genai.GenerativeModel("gemini-flash-latest")

response = model.generate_content(
    [
        "Explain what AI is"
    ],
    generation_config={
        "temperature": 0.7, # Controls randomness; lower values make output more deterministic.
        "max_output_tokens": 100 # Sets the maximum number of tokens to generate.
    }
)

print("\n--- ANSWER ---")
display(Markdown(response.text))

#Part 2 - Open Source Models

### HuggingFace Transformers

HuggingFace Transformers is a powerful library that provides thousands of pre-trained models for various tasks in Natural Language Processing (NLP), Computer Vision, and Audio. It allows you to quickly download and use state-of-the-art models for tasks like text generation, sentiment analysis, translation, and more. It supports popular deep learning frameworks like PyTorch, TensorFlow, and JAX.

This section will explore how to use different `pipeline` functions from the Transformers library for various NLP tasks with open-source models.

#Install Transformers
The Transformers library is already installed Google Colab. You just have to import it.

In [ ]:
import transformers

Models available on HuggingFace: [Models](https://huggingface.co/models)

[Resource](https://huggingface.co/learn/llm-course/chapter1/1?fw=pt)


### Understanding the `pipeline()` Function

The `pipeline()` function from the HuggingFace Transformers library is a high-level API designed to make it easy to use pre-trained models for common NLP tasks. It abstracts away the complexities of tokenization, model loading, and post-processing, allowing you to quickly get results with just a few lines of code.

When you use `from transformers import pipeline`, you're importing this versatile function. It acts as a wrapper around models, making them accessible for specific tasks like text generation, sentiment analysis, or summarization, without needing to delve into the intricate details of each model's architecture or how to prepare the input data for it.

#Pipelines

The `pipeline()` function connects a model with its necessary preprocessing and postprocessing steps, allowing us to directly input any text and get an intelligible answer.

There are three main steps involved when you pass some text to a pipeline:

1. The text is preprocessed into a format the model can understand.
2. The preprocessed inputs are passed to the model.
3. The predictions of the model are post-processed, so you can make sense of them.



Some of the currently available pipelines are:

* feature-extraction (get the vector representation of a text)

* fill-mask

* ner (named entity recognition)

* question-answering

* sentiment-analysis

* summarization

* text-generation

* translation

* zero-shot-classification

In [ ]:
from transformers import pipeline

In [ ]:
from IPython.display import display, Markdown

### Text Generation with Open-Source Models

The `text-generation` pipeline allows you to generate new text based on a given prompt. You can specify different pre-trained models from the HuggingFace Hub.

#### GPT-2

*   `generator = pipeline('text-generation', model='gpt2')`: This initializes a text generation pipeline using the `gpt2` model. GPT-2 is a widely recognized transformer-based language model by OpenAI, known for its ability to generate coherent and diverse text.
*   The `max_length` parameter controls the maximum number of tokens in the generated output, and `num_return_sequences` specifies how many different output sequences should be generated.

#### LiquidAI

*   `generator = pipeline("text-generation", model="LiquidAI/LFM2-2.6B-Exp")`: This initializes another text generation pipeline, but this time using the `LiquidAI/LFM2-2.6B-Exp` model. This demonstrates how you can easily switch between different models available on HuggingFace by just changing the `model` parameter. This particular model is an experimental one from LiquidAI, showcasing the diversity of available open-source models.

###GPT2

In [ ]:
generator = pipeline('text-generation', model='gpt2')

In [ ]:
output = generator(
    "When you are on a diet",
    max_length=50,
    num_return_sequences=1,
)
output

In [ ]:
# Extract the generated text from the list of dictionaries
generated_texts = [item['generated_text'] for item in output]
# Join the generated texts into a single string
display(Markdown("\n".join(generated_texts)))

###LiquidAI

In [ ]:
generator = pipeline("text-generation", model="LiquidAI/LFM2-2.6B-Exp")

In [ ]:
output = generator(
    "When you are on a diet",
    max_length=50,
)

# Extract the generated text from the list of dictionaries
generated_texts = [item['generated_text'] for item in output]
# Join the generated texts into a single string
display(Markdown("\n".join(generated_texts)))

### Sentiment Analysis

Sentiment analysis is the task of classifying the emotional tone of text, typically as positive, negative, or neutral. The `sentiment-analysis` pipeline is perfect for this.

*   `classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")`: This line initializes a sentiment analysis pipeline. The model specified, `distilbert-base-uncased-finetuned-sst-2-english`, is a DistilBERT model fine-tuned on the SST-2 dataset, which is designed for sentiment classification. It's a smaller, faster version of BERT, making it efficient for such tasks.
*   When you pass text to the `classifier`, it returns a list of dictionaries, each containing the predicted `label` (e.g., 'POSITIVE', 'NEGATIVE') and a `score` indicating the confidence of that prediction.

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")


In [ ]:
classifier("This is quite difficult")

### Batch Processing for Sentiment Analysis (Give this a try)

This section demonstrates how to apply the sentiment analysis pipeline to multiple pieces of text efficiently, often referred to as batch processing. Instead of processing one sentence at a time, you can pass a list of texts to the pipeline.

*   `import pandas as pd`: Imports the pandas library, which is commonly used for data manipulation, especially with tabular data.
*   `df = pd.read_csv('...', nrows=100)`: Reads the first 100 rows of the `sms_spam.csv` dataset into a pandas DataFrame. This dataset likely contains text messages that can be analyzed for sentiment.
*   `results = classifier(df['text'].tolist())`: This is the key line for batch processing. It takes the 'text' column from the DataFrame, converts it into a list, and passes this list directly to the `classifier` pipeline. The pipeline processes all texts in the list simultaneously.
*   The subsequent lines `labels = [item['label'] for item in results]` and `scores = [item['score'] for item in results]` extract the sentiment labels and scores from the results.
*   `df['sentiment_label'] = labels` and `df['sentiment_score'] = scores`: These lines add the extracted labels and scores as new columns to the DataFrame, associating each text message with its predicted sentiment.

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/RDGopal/IB9LQ0-GenAI/main/Data/sms_spam.csv',nrows=100)

In [ ]:
df

In [ ]:
try:
    results = classifier(df['text'].tolist())
except Exception as e:
    print(f"An error occurred: {e}")

labels = [item['label'] for item in results]
scores = [item['score'] for item in results]

df['sentiment_label'] = labels
df['sentiment_score'] = scores

In [ ]:
df

### Zero-Shot Classification

Zero-shot classification is a powerful NLP technique where a model can classify text into categories it hasn't explicitly been trained on. Instead, it classifies based on a natural language description of the categories.

*   `classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")`: This initializes a zero-shot classification pipeline using the `facebook/bart-large-mnli` model. This model is fine-tuned for natural language inference (MNLI), which allows it to understand the relationship between a premise (your text) and a hypothesis (your candidate labels).
*   When calling the `classifier`, you provide both the `sequence` (the text to be classified) and a list of `candidate_labels`. The model then determines how likely the sequence belongs to each of the provided labels, even if it has never seen these specific labels during its training for this task. The output includes the `sequence`, `labels`, and corresponding `scores`.

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
classifier(
    "This is a discussion about world history",
    candidate_labels=["education", "politics", "business"],
)

### Fill-Mask

The `fill-mask` pipeline is used to predict missing words (masked tokens) in a sentence. This is a common task used to pre-train language models.

*   `unmasker = pipeline("fill-mask", model = "distilroberta-base")`: This initializes a fill-mask pipeline using the `distilroberta-base` model. DistilRoBERTa is a distilled version of RoBERTa, which itself is an optimized BERT model. It's designed for efficiency while retaining much of the performance.
*   When you pass a sentence with a `<mask>` token to the `unmasker`, the model predicts the most probable words that could fill that mask. The `top_k` parameter specifies how many of the top predictions you want to see. The output provides the `score` (confidence), `token` (ID), `token_str` (the predicted word), and the `sequence` with the mask filled.

In [ ]:
unmasker = pipeline("fill-mask", model = "distilroberta-base")

In [ ]:
unmasker("This course will teach you all about <mask> models.", top_k=5)

### Named Entity Recognition (NER)

Named Entity Recognition (NER) is the task of identifying and classifying named entities (like persons, organizations, locations, dates, etc.) in text. The `ner` pipeline helps extract these entities.

*   `ner = pipeline("ner", grouped_entities=True, model="dbmdz/bert-large-cased-finetuned-conll03-english")`: This initializes an NER pipeline. The `dbmdz/bert-large-cased-finetuned-conll03-english` model is a BERT-based model fine-tuned on a dataset (CoNLL-2003) specifically for NER tasks in English.
*   `grouped_entities=True` is a useful parameter that combines sub-word tokens belonging to the same entity into a single entity. For example, "New York" might be tokenized as two separate tokens but would be grouped as a single 'LOC' (location) entity.
*   The output shows the `entity_group` (e.g., 'PER' for Person, 'ORG' for Organization, 'LOC' for Location), a `score` (confidence), the `word` itself, and its `start` and `end` character positions in the original text.

In [ ]:
ner = pipeline("ner", grouped_entities=True, model="dbmdz/bert-large-cased-finetuned-conll03-english")

In [ ]:
ner("My name is Ram and I work at WBS in Coventry.")

### Summarization

Text summarization is the task of creating a shorter, coherent, and fluent summary of a longer text document. The `summarization` pipeline helps condense information.

*   `summarizer = pipeline("summarization", model = "sshleifer/distilbart-cnn-12-6")`: This initializes a summarization pipeline using the `sshleifer/distilbart-cnn-12-6` model. This model is a DistilBART variant fine-tuned for summarization. BART (Bidirectional and Auto-Regressive Transformers) is known for its effectiveness in generation tasks like summarization.
*   You pass a longer piece of text to the `summarizer`, and it returns a dictionary containing the `summary_text`.

In [ ]:
summarizer = pipeline("summarization", model = "sshleifer/distilbart-cnn-12-6")

In [ ]:
output = summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of
    graduates in traditional engineering disciplines such as mechanical, civil,
    electrical, chemical, and aeronautical engineering declined, but in most of
    the premier American universities engineering curricula now concentrate on
    and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely supporting increasingly complex scientific
    developments. While the latter is important, it should not be at the expense
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other
    industrial countries in Europe and Asia, continue to encourage and advance
    the teaching of engineering. Both China and India, respectively, graduate
    six and eight times as many traditional engineers as does the United States.
    Other industrial countries at minimum maintain their output, while America
    suffers an increasingly serious decline in the number of engineering graduates
    and a lack of well-educated engineers.
"""
)
display(Markdown(output[0]['summary_text']))

### Translation

The `translation` pipeline allows you to translate text from one language to another. You need to specify the source and target languages in the pipeline name.

*   `trans = pipeline("translation_en_to_fr", model = "t5-base")`: This initializes a translation pipeline specifically for translating from English (`en`) to French (`fr`). The `t5-base` model is a T5 (Text-to-Text Transfer Transformer) model, which frames all NLP problems as text-to-text tasks, making it versatile for translation, summarization, and more.
*   When you pass an English sentence to `trans`, it returns a list of dictionaries, each containing the `translation_text` in French.

In [ ]:
trans = pipeline("translation_en_to_fr", model = "t5-base")

In [ ]:
y = trans("good evening")
list(y)



---

